In [ ]:
import pandas as pd
from functools import reduce

Este código tiene el de consolidar datos de los EE.FF. de las siguientes instituciones financieras: Bancas Múltiples, Empresas Financieras y Cajas Rurales, Cajas Municipales.
Al consolidarlo podemos realizar un mejor análisis de la evolución de sus cuentas. 

### Consolidado de todas las bancas por tipo de institución financiera

In [ ]:

# Definir las cuentas importantes
cuentas_importantes = {
    'DISPONIBLE', 'INVERSIONES NETAS DE PROVISIONES ', 'CRÉDITOS NETOS DE PROVISIONES ', 'CUENTAS POR COBRAR NETAS DE PROVISIONES', 'TOTAL ACTIVO', 'OBLIGACIONES CON EL PÚBLICO',
    'DEPÓSITOS DEL SISTEMA FINANCIERO Y ORGANISMOS INTERNACIONALES', 'ADEUDOS Y OBLIGACIONES  FINANCIERAS', 'TOTAL PASIVO', 'PATRIMONIO', 'TOTAL PASIVO Y PATRIMONIO',
    'INGRESOS FINANCIEROS ', 'GASTOS FINANCIEROS ', 'MARGEN FINANCIERO NETO', 'RESULTADO NETO DEL EJERCICIO', '   Atrasados'
}

# Ruta del archivo de entrada
archivo_entrada = "banca_multiple.xlsx"

# Leer todas las hojas del archivo de Excel
hojas = pd.read_excel(archivo_entrada, sheet_name=None, header=None)

# Crear un DataFrame vacío para el consolidado
consolidado = pd.DataFrame()

# Procesar cada hoja
for nombre_hoja, df in hojas.items():
    # Extraer las fechas de la fila 4 (índice 3)
    fechas = df.iloc[4, 1:].tolist()  # Con el excel de rural cambiar 4 por 3
    
    # Propagar las fechas hacia la derecha
    fechas_propagadas = []
    fecha_actual = None
    for fecha in fechas:
        if pd.notna(fecha):  # Si la celda tiene una fecha
            fecha_actual = fecha
        fechas_propagadas.append(fecha_actual)
    
    # Filtrar solo las columnas donde la fila siguiente contenga "Total"
    columnas_total = [0] + [i+1 for i in range(len(fechas_propagadas)) if "TOTAL" in str(df.iloc[5, i+1])]  # Con el excel de rural cambiar 5 por 4
    df = df.iloc[:, columnas_total]
    
    # Asignar las fechas propagadas solo a las columnas seleccionadas
    fechas_filtradas = ["Cuenta"] + [fechas_propagadas[i-1] for i in columnas_total[1:]]
    df.columns = fechas_filtradas
    
    # Filtrar las filas que corresponden a las cuentas importantes (a partir de la fila 5)
    df_filtrado = df[df.iloc[:, 0].isin(cuentas_importantes)]
    
    # Agregar una columna con el nombre de la banca
    df_filtrado.insert(0, "entidad", nombre_hoja)
    
    # Concatenar al DataFrame consolidado
    consolidado = pd.concat([consolidado, df_filtrado], ignore_index=True)

# Ruta del archivo de salida
archivo_salida = "consolidado_banca_multiple.xlsx"

# Guardar el consolidado en un nuevo archivo de Excel
consolidado.to_excel(archivo_salida, index=False)

print(f"Consolidación completada. Los datos se han guardado en '{archivo_salida}'.")

Consolidación completada. Los datos se han guardado en 'consolidado_33.xlsx'.


** Utilizar si se tiene datos de fechas pasadas a 2013

In [ ]:
# Lista de archivos a procesar
archivos = ["consolidado_banca_multiple_04.xlsx", "consolidado_banca_multiple_08.xlsx", "consolidado_banca_multiple_13.xlsx", 'consolidado_banca_multiple_19.xlsx']

# Leer y almacenar los DataFrames
dataframes = [pd.read_excel(archivo, sheet_name=0) for archivo in archivos]

# Hacer el merge progresivamente asegurando que las fechas se unan correctamente
df_final = reduce(lambda left, right: pd.merge(left, right, on=["Banca_multiple", "Cuenta"], how="outer"), dataframes)

# Renombrar columnas
df_final.rename(columns={"Banca_multiple": "entidad", "Cuenta": "cuenta"}, inplace=True)

# Agregar la nueva columna 'tipo_institucion' al inicio
df_final.insert(0, "tipo_institucion", "Empresas Financieras")

# Ordenar las columnas de fechas
columnas_fijas = ["tipo_institucion", "entidad", "cuenta"]
columnas_fechas = sorted([col for col in df_final.columns if col not in columnas_fijas], key=lambda x: pd.to_datetime(x, format="%m-%Y", errors="coerce"))

# Reorganizar el DataFrame con las fechas ordenadas
df_final = df_final[columnas_fijas + columnas_fechas]

# Guardar el archivo final con la estructura corregida
df_final.to_excel("consolidado_banca_multiple.xlsx", index=False)

print(f"Consolidación completada.")

### Consolidado Final

In [ ]:
# Lista de archivos a consolidar
archivos = ["consolidado_banca_multiple.xlsx", "consolidado_financieras.xlsx", "consolidado_rural.xlsx", "consolidado_municipal.xlsx"]

# Leer y combinar todos los DataFrames
df_consolidado = pd.concat([pd.read_excel(archivo) for archivo in archivos], ignore_index=True)

# Guardar el archivo consolidado
df_consolidado.to_excel("data_final.xlsx", index=False)

print(f"Consolidación completada.")

### Transformación a CSV (Para analizarlo más fácil)

In [ ]:
# Importar el archivo consolidado
archivo = "data_final.xlsx"
df = pd.read_excel(archivo)

# Transformar de formato ancho (wide) a largo (long)
df_long = df.melt(id_vars=["tipo_institucion", "entidad", "cuenta"], 
                  var_name="fecha", 
                  value_name="monto")

# Reorganizar en formato final para Stata
df_final = df_long.pivot_table(index=["tipo_institucion", "entidad", "fecha"], 
                               columns="cuenta", 
                               values="monto").reset_index()

# Renombrar columnas si es necesario
df_final.columns.name = None  # Eliminar el nombre de la columna de índices

# Guardar en formato CSV para Stata
df_final.to_csv("data_csv.csv", index=False)

print("Transformación completa. Archivo listo para Analizar.")